## Database

In [142]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

Databases de cada Local para facilitar a limpeza e padronização dos dataframes

In [143]:
#Lendo a base de dados
#df1=pd.read_csv('Focos_2018-12-31_2019-12-31.csv', low_memory=False)
#df2=pd.read_csv('Focos_2020-01-01_2020-10-04.csv', low_memory=False)

In [144]:
#frames=[df1,df2]
#df = pd.concat(frames)
df=pd.read_csv('Focos_2018-10-03_2019-10-03.csv', low_memory=False)

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
#demonstrando dados iniciais
df.head()

In [ ]:
#Retirando variáveis desnecessárias e separando a data e a hora
df[['data','hora']] = df.datahora.str.split(" ",expand=True,)
df_novo=df.drop(["datahora","satelite","bioma","municipio","estado","frp"], axis=1)
#df_novo=df_novo[(df_novo['riscofogo']>0)]
#df_novo=df_novo[(df_novo['diasemchuva']<0)]
df.novo=df_novo.dropna(how='any', inplace=True)
indexNames = df_novo[ (df_novo['diasemchuva'] < 0) | (df_novo['riscofogo'] < 0) ].index
df_novo.drop(indexNames , inplace=True)
#df_novo=df_novo.replace(-999, 0)
df_novo.head()

In [ ]:
df_novo.riscofogo.value_counts()

In [ ]:
f,ax = plt.subplots(figsize = (10,10))
sns.heatmap(df_novo.corr(),annot= True, fmt='.1f')

In [ ]:
fig=plt.figure(figsize=(10,10))

#Riscofogo X Precipitação
plt.subplot(221)
plt.scatter(df_novo.riscofogo,df_novo.precipitacao)
plt.title("Riscofogo X Precipitação")
plt.ylabel("Precipitação")
plt.xlabel("Risco de Fogo")

#Riscofogo X Dias Sem chuva
plt.subplot(222)
plt.scatter(df_novo.riscofogo,df_novo.diasemchuva)
plt.title("Riscofogo X Dias Sem chuva")
plt.ylabel("Dias sem chuva")
plt.xlabel("Risco de Fogo")

#Riscofogo X Latitude
plt.subplot(223)
plt.scatter(df_novo.riscofogo,df_novo.latitude)
plt.title("Riscofogo X Latitude")
plt.ylabel("Latitude")
plt.xlabel("Risco de Fogo")

#Riscofogo X Longitude
plt.subplot(224)
plt.scatter(df_novo.riscofogo,df_novo.longitude)
plt.title("Riscofogo X Longitude")
plt.ylabel("Longitude")
plt.xlabel("Risco de Fogo")

plt.show()

In [ ]:
fig=plt.figure(figsize=(15,10))

plt.subplot(221)
stats.probplot(df_novo.riscofogo, plot = plt )
plt.title("Probplot:Risco de Fogo")

plt.subplot(222)
stats.probplot(df_novo.diasemchuva, plot = plt )
plt.title("Probplot:Dias Sem Chuva")

plt.subplot(223)
stats.probplot(df_novo.latitude, plot = plt )
plt.title("Probplot:Latitude")

plt.subplot(224)
stats.probplot(df_novo.longitude, plot = plt )
plt.title("Probplot:Longitude")

In [ ]:
#Salvando base de dados como CSV
#df_novo.to_csv('BaseDeDados.csv')